In [109]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [110]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = MNIST('data', train=True, transform=train_transforms, download=True)
test_dataset = MNIST('data', train=False, transform=test_transforms, download=True)

batch_size = 100
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [111]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 10)
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = self.dropout(F.leaky_relu(self.fc1(x)))
        x = self.dropout(F.leaky_relu(self.fc2(x)))
        x = self.dropout(F.leaky_relu(self.fc3(x)))
        x = self.dropout(F.leaky_relu(self.fc4(x)))
        return F.log_softmax(self.fc5(x), dim=1)


In [113]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.001)

def train(model, train_loader, criterion, optimizer, device):
    model.train()
    mean_loss = 0.0
    for data, labels in train_loader:
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        mean_loss += loss.item()
    mean_loss /= len(train_loader)
    return mean_loss

epochs = 25

def evaluate(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

for epoch in range(epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    accuracy = evaluate(model, test_loader, device)
    print(f"Epoch {epoch+1}/{epochs}, Training Loss: {train_loss:.4f}, Accuracy: {accuracy:.2f}%")

Epoch 1/25, Training Loss: 0.6354, Accuracy: 93.19%
Epoch 2/25, Training Loss: 0.2609, Accuracy: 95.71%
Epoch 3/25, Training Loss: 0.2003, Accuracy: 95.55%
Epoch 4/25, Training Loss: 0.1731, Accuracy: 96.55%
Epoch 5/25, Training Loss: 0.1545, Accuracy: 96.94%
Epoch 6/25, Training Loss: 0.1372, Accuracy: 96.95%
Epoch 7/25, Training Loss: 0.1283, Accuracy: 97.35%
Epoch 8/25, Training Loss: 0.1195, Accuracy: 97.21%
Epoch 9/25, Training Loss: 0.1088, Accuracy: 97.41%
Epoch 10/25, Training Loss: 0.1081, Accuracy: 97.80%
Epoch 11/25, Training Loss: 0.0995, Accuracy: 97.67%
Epoch 12/25, Training Loss: 0.0977, Accuracy: 97.51%
Epoch 13/25, Training Loss: 0.0893, Accuracy: 97.85%
Epoch 14/25, Training Loss: 0.0871, Accuracy: 97.99%
Epoch 15/25, Training Loss: 0.0806, Accuracy: 97.78%
Epoch 16/25, Training Loss: 0.0801, Accuracy: 97.61%
Epoch 17/25, Training Loss: 0.0766, Accuracy: 97.85%
Epoch 18/25, Training Loss: 0.0710, Accuracy: 97.94%
Epoch 19/25, Training Loss: 0.0721, Accuracy: 97.96%
Ep

In [117]:
model.eval()
correct = 0
total = 0
predictions = []
with torch.no_grad():
    for data, labels in test_loader:
        data, labels = data.to(device), labels.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        predictions.extend(predicted.cpu().numpy())

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')

# Create submission file
submission = pd.DataFrame({
    'ID': np.arange(len(predictions)),
    'target': predictions
})
submission.to_csv('submission.csv', index=False)

Accuracy: 98.10%
Current Working Directory: /Users/bughy/Desktop/Master/NeuralNetworks/tema3
File 'submission.csv' created successfully.
